In [1]:
pip uninstall -r requirements.txt -y

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip uninstall langchain -y

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -r requirements.txt

  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached httpcore-1.0.3-py3-none-any.whl.metadata (20 kB)
  Using cached httpx-0.26.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached asyncio-3.4.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached fastapi-0.109.0-py3-none-any.whl.metadata (24 kB)
  Using cached langchain-0.1.6-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached langchain_core-0.1.23-py3-none-any.whl.metadata (6.0 kB)
  Us

ERROR: Cannot install -r requirements.txt (line 16) and tiktoken==0.5.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
! pip install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

In [ ]:
! pip install pypdf faiss-cpu

In [55]:
from getpass import getpass
HUGGINGFACE_API_TOKEN = getpass()

In [56]:
import os
os.environ["HUGGINGFACE_API_TOKEN"]=HUGGINGFACE_API_TOKEN

In [8]:
!pip install langchain==0.1.6
!pip install langchain_community==0.0.19
!pip install langchain_core==0.1.23

  Using cached langchain-0.1.6-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl.metadata (8.4 kB)
  Using cached langchain_core-0.1.38-py3-none-any.whl.metadata (6.0 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     -------------------------------------- - 81.9/85.1 kB 2.2 MB/s eta 0:00:01
     ---------------------------------------- 85.1/85.1 kB 1.6 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.30-py3-none-any.whl.metadata (8.4 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_community-0.0.28-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_commu

In [52]:
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [53]:
question = "What is the capital of France?"
template = """Question: {question}
                Answer: Let's think step by step."""

prompt = PromptTemplate(template=template,input_variables=["question"])

In [54]:
repo_id = "McGill-NLP/flan-t5-large-weblinx"

In [12]:
llm = HuggingFaceHub(repo_id=repo_id,huggingfacehub_api_token=os.environ["HUGGINGFACE_API_TOKEN"])
llm_chain = LLMChain(llm=llm,prompt=prompt,llm_kwargs={"temperature":0,"max_length":400})

print(llm_chain.run(question))

c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Question: What is the capital of France?
                Answer: Let's think step by step.swer is Paris.


In [13]:
import langchain
print(langchain.__version__)

0.1.6


In [14]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/AC_AR2023_EN.pdf")
pages = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)

docs = text_splitter.split_documents(pages[3:5])

In [28]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.from_documents(docs,embeddings)
query = "what is the Message from the President and Chief Executive Officer"
result_simi = db.similarity_search(query,k=2)

In [29]:
source_knowledge = "\n".join([x.page_content for x in result_simi])

In [30]:
augmented_propmt = """
Using the contexts below, answer the query.

contexts:
{source_knowledge}

query:
{query}
"""

In [31]:
prompt = PromptTemplate(template=augmented_propmt,input_variables=["source_knowledge","query"])

llm_chain = LLMChain(llm=llm,prompt=prompt,llm_kwargs={"temperature":0,"max_length":400})

print(llm_chain.run({"source_knowledge":source_knowledge,"query":query}))


Using the contexts below, answer the query.

contexts:
Chief Executive Officer
We are in the business of journeys and making 
meaningful connections. As we reflect on 
Air Canada’s 2023 performance, I am pleased to 
share the accomplishments and progress in a year 
full of challenges and opportunities. We are proud
of our achievements and remain committed to 
creating long-term value for all our stakeholders.

query:
what is the Message from the President and Chief Executive Officer



In [34]:
augmented_propmt_2 = f"""Using the contexts below, answer the query.
query: {query}
contexts: {source_knowledge}

"""


In [35]:
print(augmented_propmt_2)

Using the contexts below, answer the query.
query: what is the Message from the President and Chief Executive Officer
contexts: Chief Executive Officer
We are in the business of journeys and making 
meaningful connections. As we reflect on 
Air Canada’s 2023 performance, I am pleased to 
share the accomplishments and progress in a year 
full of challenges and opportunities. We are proud
of our achievements and remain committed to 
creating long-term value for all our stakeholders.




In [46]:
docs_part2 = text_splitter.split_documents(pages[32:85])

In [47]:
qa_generation_template = """You are an assistant that helpe strategists to find information about pricing stratgy, Ancillary Revenue and all metrics.
You should focus on the part: 2023 Management's Discussion and Analysis of Results of Operations and Financial Condition
You should show all the key takeaways, enhanced with specific metrics and detailed information.

You should show the result in well formed stucture, which is easy to understand.
For instance:
Pricing Strategy:
1.Yield Management: The airline experienced a year-over-year increase in yields driven by a strong pricing environment. 
This was supported by strong demand for premium products across all markets, indicating effective revenue management and pricing strategies .
2.Market-Specific Revenue Changes: Specific details on passenger revenues by geographic region for the fourth quarter of 2023 revealed substantial increases, especially in the Pacific and Atlantic regions, which saw 48.1% and 14.8% increases in passenger revenues, respectively. 
Such growth rates underscore the impact of strategic pricing and capacity management in response to market demand .

You answer should try to show all the related metrics, just like the example above.

Query Results:
{context}

Question:
{question}

If the provided information is empty, say you don't know the answer.
Empty information looks like this: []

If the information is not empty, you must provide an answer using the results.
  """

In [48]:
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)

In [49]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

#embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.from_documents(docs_part2,embeddings)
question = "how is the Air Transat pricing strategy in 2023?"
result_simi = db.similarity_search(query,k=5)

In [50]:
context = "\n".join([x.page_content for x in result_simi])

In [57]:
llm = HuggingFaceHub(repo_id=repo_id,huggingfacehub_api_token=os.environ["HUGGINGFACE_API_TOKEN"])
llm_chain = LLMChain(llm=llm,prompt=qa_generation_prompt,llm_kwargs={"temperature":0,"max_length":400})
print(llm_chain.run({"context":context,"question":question}))

You are an assistant that helpe strategists to find information about pricing stratgy, Ancillary Revenue and all metrics.
You should focus on the part: 2023 Management's Discussion and Analysis of Results of Operations and Financial Condition
You should show all the key takeaways, enhanced with specific metrics and detailed information.

You should show the result in well formed stucture, which is easy to understand.
For instance:
Pricing Strategy:
1.Yield Management: The airline experienced a year-over-year increase in yields driven by a strong pricing environment. 
This was supported by strong demand for premium products across all markets, indicating effective revenue management and pricing strategies .
2.Market-Specific Revenue Changes: Specific details on passenger revenues by geographic region for the fourth quarter of 2023 revealed substantial increases, especially in the Pacific and Atlantic regions, which saw 48.1% and 14.8% increases in passenger revenues, respectively. 
Such